In [8]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
%matplotlib inline

In [9]:
### how to make from permatation matrix readable array
def perm_array(A):
    p_a = np.array((A.shape[0]))
    p_a = np.argmax(A, axis = 1)
    return p_a

In [3]:
n = 4 #nmber of sampling points   
deg = 3 # Degree of polynomiapaired plu Last Checkpoint: 22 minutes ago (unsaved changes)l          
interval = [-3, 3]  # interval of x
x = np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1

In [4]:
### generating matrix with special structure
b = Gen_1D_coupled(p_size, x, debug=True)
b

number of points(l) = 4,  number of monoms(n_size) = 4


array([[   1.,   -3.,   17.,  -99.],
       [   0.,    1.,  -12.,  105.],
       [   1.,   -1.,    1.,   -1.],
       [   0.,    1.,   -4.,    9.],
       [   1.,    1.,    1.,    1.],
       [   0.,    1.,    4.,    9.],
       [   1.,    3.,   17.,   99.],
       [   0.,    1.,   12.,  105.]])

In [5]:
p,l,u,q = pluq_ids(b)

[[   1.    0.]
 [  -3.    1.]
 [  17.  -12.]
 [ -99.  105.]]
[[ 1.  0.]
 [-1.  1.]
 [ 1. -4.]
 [-1.  9.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 1.  4.]
 [ 1.  9.]]
[[   1.    0.]
 [   3.    1.]
 [  17.   12.]
 [  99.  105.]]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 2, 3)
('maxvol 2x2 submatrix', array([[  17.,  -12.],
       [ -99.,  105.]]))
('with det = ', 596.99999999999955)
pivoting and permutations start...
[[  17.   -3.    1.  -99.]
 [ -12.    1.    0.  105.]
 [   1.   -1.    1.   -1.]
 [  -4.    1.    0.    9.]
 [   1.    1.    1.    1.]
 [   4.    1.    0.    9.]
 [  17.    3.    1.   99.]
 [  12.    1.    0.  105.]]
[[  17.           -3.            1.          -99.        ]
 [   0.           -1.11764706    0.70588235   35.11764706]
 [   0.           -0.82352941    0.94117647    4.82352941]
 [   0.            0.29411765    0.23529412  -14.29411765]
 [   0.            1.17647059    0.94117647    6.82352941]
 [   0.            1.70588235   

In [6]:
### here we can see that in the first 4 rows blocked structure was saved, i.e. func-deriv, func-deriv
perm_array(p)

array([0, 1, 6, 7, 4, 5, 2, 3])

In [ ]:
### maxvol preprocessing
ids = np.dot(p,np.dot(b,q))[:p_size]
A = np.dot(p,np.dot(b,q))

temp = np.dot(A,np.linalg.inv(ids))

In [ ]:
### here we are checking all pairs under Identity matrix and try to find 2x2 block with det>1+tol
tol = 0.05
n = temp.shape[0]
m = temp.shape[1]
for k in range(m,n,2):
        pair = np.concatenate((temp[k,:],temp[k+1,:])).reshape(2,m).T
        piv,_ = maxvol(pair)
        piv = np.sort(piv)
        print pair, pair[piv]
        if (np.abs(np.linalg.det(pair[piv])) > (1 + tol)):
            print('exist!')

# RGV

In [10]:
mat_test = np.array([ 
    [50,  0 , 0,0,0, 1],
    [0, 0, 30,  0 ,0, 3],
    [0,0, 0, 20,  0 , 11],
    [0, 40,  0 ,0,0, 2],
    [0,0, 0,0,  10,  12],
    [0,0,0, 0,0, 50]
], dtype=float)
print (mat_test)
print ()

[[ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
()


In [11]:
p,l,u,q, info = pluq_ids(mat_test, debug=True)

correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 0, 2)
('maxvol 2x2 submatrix', array([[ 50.,   0.],
       [  0.,  30.]]))
('with det = ', 1500.0)
pivoting and permutations start...
after 1st pivot
[[ 50.   0.   0.   0.   0.   1.]
 [  0.   0.  30.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.  40.   0.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
after 2nd pivot
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  30.   0.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  40.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
Elimination starts
after 1st elimination
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  30.   0.   0.   0.   3.]
 [  0.   0.   0.  20.   0.  11.]
 [  0.   0.  40.   0.   0.   2.]
 [  0.   0.   0.   0.  10.  12.]
 [  0.   0.   0.   0.   0.  50.]]
after 2nd ilimination
[[ 50.   0.   0.   0.   0.   1.]
 [  0.  30.   0.   0.   0.   3.]
 [  0.   0.   

In [12]:
mat_test_3 = np.array([ 
    [20,0, 0, 2, 0 , 11],
    [0, 10,  0 ,0,0.043, 2],    
    [0,  0 , 30,0,15, 1],
    [0, 0, 0,  10 ,0, 0],
    [0,0, 20,0,  90,  12],
    [0,0,0, 0,0, 1900]
], dtype=float)
print (mat_test_3.astype(int))
print ()

[[  20    0    0    2    0   11]
 [   0   10    0    0    0    2]
 [   0    0   30    0   15    1]
 [   0    0    0   10    0    0]
 [   0    0   20    0   90   12]
 [   0    0    0    0    0 1900]]
()


In [13]:
p,l,u,q, info = pluq_ids(mat_test_3, debug=True)

correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 4, 5)
('column coordinates:', 4, 5)
('maxvol 2x2 submatrix', array([[   90.,     0.],
       [   12.,  1900.]]))
('with det = ', 171000.00000000003)
pivoting and permutations start...
after 1st pivot
[[  9.00000000e+01   0.00000000e+00   2.00000000e+01   0.00000000e+00
    0.00000000e+00   1.20000000e+01]
 [  4.30000000e-02   1.00000000e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   2.00000000e+00]
 [  1.50000000e+01   0.00000000e+00   3.00000000e+01   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+01
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   2.00000000e+00
    2.00000000e+01   1.10000000e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.90000000e+03]]
after 2nd pivot
[[  9.00000000e+01   1.20000000e+01   2.00000000e+01   0.00000000e+00
    0.0

# RGV 2
### Test for linear dependency. 1st thought about pullbacks

In [14]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

## Check, that matrix is good

In [3]:
np.linalg.matrix_rank(mat_test)

6

In [4]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

-7999999.9999999702

In [15]:
p,l,u,q, info = pluq_ids(mat_test, debug=False)
print info

[0 1]


# Another Error 
### That was about columns swap. 
If piv[0] == j, piv[1] != j+1

In [16]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [18]:
p,l,u,q, info = pluq_ids(mat_test_5, debug=True)

diag case
[[  0.  17.]
 [ 14.  17.]]
('on the', 0, 'slice')
('best row block is', 6, 7)
('column coordinates:', 0, 4)
('maxvol 2x2 submatrix', array([[  0.,  14.],
       [ 17.,  17.]]))
('with det = ', 237.99999999999986)
pivoting and permutations start...
after 1st pivot
[[ 17.   7.   7.   6.   0.   9.]
 [ 17.  15.   5.   6.  16.  15.]
 [ 12.   0.   7.   7.  15.  11.]
 [  6.   4.   6.  13.  12.   5.]
 [  3.   8.  10.  16.  19.  16.]
 [  1.   3.   5.   1.  10.   4.]
 [ 19.   7.  10.  14.  18.  12.]
 [ 17.   7.   9.  16.  14.   5.]
 [  9.  18.  19.   3.  18.  12.]
 [  0.   7.  16.   1.  12.   2.]]
after 2nd pivot
[[ 17.   0.   7.   6.   7.   9.]
 [ 17.  14.   9.  16.   7.   5.]
 [ 12.  15.   7.   7.   0.  11.]
 [  6.  12.   6.  13.   4.   5.]
 [  3.  19.  10.  16.   8.  16.]
 [  1.  10.   5.   1.   3.   4.]
 [ 19.  18.  10.  14.   7.  12.]
 [ 17.  16.   5.   6.  15.  15.]
 [  9.  18.  19.   3.  18.  12.]
 [  0.  12.  16.   1.   7.   2.]]
Elimination starts
after 1st elimination
[[ 17. 

## Yet another bug
About feature of maxvol's performance. It returns error if we have a long pair consisted from more then 2 elements each with rank < 2

In [19]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [20]:
p,l,u,q, info = pluq_ids(mat_test_6, debug=True)

correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', 3, 5)
('maxvol 2x2 submatrix', array([[ 18.,   2.],
       [  6.,  16.]]))
('with det = ', 275.99999999999989)
pivoting and permutations start...
after 1st pivot
[[ 18.  14.  18.  17.   9.   6.]
 [  1.  19.  16.  14.   4.   7.]
 [ 11.  13.   4.   3.   6.  11.]
 [  2.   8.   8.   7.  16.  16.]
 [ 10.  19.   3.  10.  18.   5.]
 [ 19.  17.   9.   1.  15.  13.]
 [  1.   1.   4.  13.  17.   9.]
 [ 11.  13.   8.  10.   3.   7.]
 [  0.   8.   6.   4.   1.   8.]
 [  9.  10.  10.   8.   1.  19.]]
after 2nd pivot
[[ 18.   6.  18.  17.   9.  14.]
 [  2.  16.   8.   7.  16.   8.]
 [ 11.  11.   4.   3.   6.  13.]
 [  1.   7.  16.  14.   4.  19.]
 [ 10.   5.   3.  10.  18.  19.]
 [ 19.  13.   9.   1.  15.  17.]
 [  1.   9.   4.  13.  17.   1.]
 [ 11.   7.   8.  10.   3.  13.]
 [  0.   8.   6.   4.   1.   8.]
 [  9.  19.  10.   8.   1.  10.]]
Elimination starts
after 1st elimination
[[ 18.           6.      

## Yet another bug ver. 2.0: Division by Zero
### This and below: again about columns swap. Solved

In [21]:
mat_test_7 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [22]:
p,l,u,q, info = pluq_ids(mat_test_7, debug=True)

correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', 3, 5)
('maxvol 2x2 submatrix', array([[-18.,  -9.],
       [ 19., -14.]]))
('with det = ', 422.99999999999983)
pivoting and permutations start...
after 1st pivot
[[-18.  14. -17.  -5.   4.  19.]
 [ -9.   0.  16.  -1.   3. -14.]
 [ -4.   0. -16.  12.  -6.  18.]
 [ 12.  -4. -17.  -9.  14.   0.]
 [  0.   2.   0.   0. -15.   0.]
 [ -7.  -5.  -8.  13.  13. -16.]
 [  7.   0.   3.  -5.  -7.   0.]
 [ 18.  -1. -10.  19.  17.  -9.]
 [-12.   6.  -5. -15.   2.   4.]
 [-14. -19. -13. -15.   0.  -1.]]
after 2nd pivot
[[-18.  19. -17.  -5.   4.  14.]
 [ -9. -14.  16.  -1.   3.   0.]
 [ -4.  18. -16.  12.  -6.   0.]
 [ 12.   0. -17.  -9.  14.  -4.]
 [  0.   0.   0.   0. -15.   2.]
 [ -7. -16.  -8.  13.  13.  -5.]
 [  7.   0.   3.  -5.  -7.   0.]
 [ 18.  -9. -10.  19.  17.  -1.]
 [-12.   4.  -5. -15.   2.   6.]
 [-14.  -1. -13. -15.   0. -19.]]
Elimination starts
after 1st elimination
[[-18.          19.      

## Yet another bug ver. 3.0: Division by Zero

In [23]:
mat_test_8 = np.array([[ -5,  -9, -13,  -2,  19,  11],
       [ 14, -17,  -8,   2, -12,   0],
       [ 17, -19,  -3,   6, -19,   1],
       [ 19,   0, -16, -19,  19,  18],
       [ -5,   2,   1,   4, -17,   1],
       [  4,   2,   7,   9, -10, -15],
       [ 19,  -9,   0, -12,   0,  -6],
       [-10,  15,  12,   2,   2,  13],
       [ 16,   0,   3, -16,  16, -15],
       [-15, -14,   9,   8,   0,  17]], dtype=float)

In [24]:
p,l,u,q, info = pluq_ids(mat_test_8, debug=True)

correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', 0, 4)
('maxvol 2x2 submatrix', array([[ 17.,  19.],
       [-19.,  19.]]))
('with det = ', 683.99999999999977)
pivoting and permutations start...
after 1st pivot
[[ 17. -19.  -3.   6. -19.   1.]
 [ 14. -17.  -8.   2. -12.   0.]
 [ -5.  -9. -13.  -2.  19.  11.]
 [ 19.   0. -16. -19.  19.  18.]
 [ -5.   2.   1.   4. -17.   1.]
 [  4.   2.   7.   9. -10. -15.]
 [ 19.  -9.   0. -12.   0.  -6.]
 [-10.  15.  12.   2.   2.  13.]
 [ 16.   0.   3. -16.  16. -15.]
 [-15. -14.   9.   8.   0.  17.]]
after 2nd pivot
[[ 17. -19.  -3.   6. -19.   1.]
 [ 19.  19. -16. -19.   0.  18.]
 [ -5.  19. -13.  -2.  -9.  11.]
 [ 14. -12.  -8.   2. -17.   0.]
 [ -5. -17.   1.   4.   2.   1.]
 [  4. -10.   7.   9.   2. -15.]
 [ 19.   0.   0. -12.  -9.  -6.]
 [-10.   2.  12.   2.  15.  13.]
 [ 16.  16.   3. -16.   0. -15.]
 [-15.   0.   9.   8. -14.  17.]]
Elimination starts
after 1st elimination
[[ 17.         -19.      